# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [ ]:
from openai import OpenAI

import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv('../../IHAI-lessons/000_lesson_data/044_llm/.env')) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [ ]:
def collect_messages(_):
    # Step 1: Retrieve the user's input from the input field.
    prompt = inp.value_input
    # Step 2: Clear the input field after retrieving the input.
    inp.value = ''
    # Step 3: Add the user's message to the 'context' list to track conversation history.
    context.append({'role':'user', 'content':f"{prompt}"})
    # Step 4: Generate a response from the chatbot based on the current conversation context.
    response = get_completion_from_messages(context)
    # Step 5: Add the bot's response to the conversation history.
    context.append({'role':'assistant', 'content':f"{response}"})
    # Step 6: Display the user's input in a new row on the UI.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    # Step 7: Display the bot's response in a new row with a background color.
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#000000'})))
    # Step 8: Return the full conversation as a column (all messages so far).
    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI library for creating interactive apps

# Initialize the Panel extension (required to run Panel in a notebook)
pn.extension()

# Initialize an empty list to collect the chat display
panels = []  # Stores each user and bot message for displaying the chat history

# Initialize the conversation context with a system message
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # Context list initialized with the "system" message,
# which sets up the bot's role and purpose.

# Step 1: Define the user input field
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
# `inp` is a text input field where users can type their messages. 
# `value="Hi"` sets a default initial message, which can be changed.
# `placeholder` provides a hint inside the input box when it's empty.

# Step 2: Create a button to submit the user's message
button_conversation = pn.widgets.Button(name="Chat!")
# `button_conversation` is a button with the label "Chat!" that the user clicks to submit their message.
# When clicked, it will trigger the chatbot to respond.

# Step 3: Bind the button to the chat function
interactive_conversation = pn.bind(collect_messages, button_conversation)
# `pn.bind` links the `collect_messages` function to `button_conversation`.
# This means every time the button is clicked, `collect_messages` runs, taking the latest input.

# Step 4: Define the dashboard layout
dashboard = pn.Column(
    inp,                                   # Add the input field to the dashboard
    pn.Row(button_conversation),           # Add the button in a row for visual layout
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
    # Display `interactive_conversation` (the function bound to the button click).
    # `loading_indicator=True` shows a loading icon while the function is processing.
    # `height=300` sets the height for this section of the dashboard.
)

# Display the dashboard interface
dashboard
# dashboard.show()  # to run it on localhost in the browser

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative

In [ ]:
import panel as pn

pn.extension()

panels = []

context = [
    {'role':'system', 'content':"""
    You are the Enchanted OrderBot, an automated service to collect orders for a magical potion shop.
    Begin by greeting the customer, then ask what potions, herbs, or enchanted items they would like.
    When the order is complete, check if they want anything else and ask if it’s for pickup or mystical delivery.
    If it’s a delivery, ask for their mystical location (e.g., "under the willow tree by the dark lake").
    Then summarize the order and confirm any special instructions before finalizing payment.
    Be sure to clarify potion sizes, effects, and any additional charms.
    Respond in a mystical, charming tone. The menu includes:
    - Potions: Healing Potion (large 15.00, medium 10.00, small 5.00),
      Invisibility Potion (20.00), Strength Elixir (large 12.00, small 6.00)
    - Enchanted Herbs: Dragonleaf (10.00 per bundle), Feyroot (8.00 per bundle), Witch’s Basil (5.00)
    - Charms and Crystals: Protection Charm (12.00), Fire Crystal (18.00), Water Crystal (15.00)
    Additional magical add-ons: Fairy Dust (+3.00), Moonlight Essence (+5.00), Wolfsbane (+2.00)
    """,}
]

inp = pn.widgets.TextInput(value = 'Greetings, wise Enchanter!', placeholder = 'Enter your request here…')

button_conversation = pn.widgets.Button(name = 'Chat!')

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(inp,
            pn.Row(button_conversation),
            pn.panel(interactive_conversation, loading_indicator = True, height = 300))

dashboard

In [ ]:
# context = [
#     {'role':'system', 'content':"""
#     You are the Pet Supplies OrderBot, an automated service to help customers order pet products.
#     Start by greeting the customer, then ask about the type of pet they’re shopping for and what items they’re looking to purchase.
#     Guide them through available categories such as food, toys, bedding, and accessories.
#     For each item, confirm the type, size, and any specific preferences or dietary needs. If they’re done ordering, 
#     ask if they need delivery or will pick it up in-store. For delivery, request their address.
#     Finally, summarize the order and ask if there are any additional instructions before confirming payment.
#     The store menu includes:
#     - Food: Dog Food (large bag 20.00, medium 15.00, small 10.00), Cat Food (canned 2.00, dry 10.00), Bird Seed (5.00 per bag)
#     - Toys: Chew Toy (5.00), Squeak Toy (3.00), Feather Wand (4.00)
#     - Bedding: Dog Bed (25.00), Cat Bed (20.00), Hamster Bedding (5.00 per pack)
#     - Accessories: Collar (8.00), Leash (10.00), Water Bowl (3.00)
#     Additional options: Gift Wrapping (+3.00), Expedited Shipping (+5.00)
#     """,}
# ]

In [ ]:
# context = [
#     {'role':'system', 'content':"""
#     You are the Office Supplies OrderBot, here to assist customers in purchasing office products for their business needs.
#     Begin by greeting the customer and asking what type of office supplies they need today.
#     Guide them through categories like stationery, technology, furniture, and cleaning supplies.
#     For each item, confirm details like quantity, color preference, and any special requests. 
#     When the customer finishes, ask if they need it delivered or if they’ll pick it up from the store. 
#     For delivery, gather the office address.
#     Finally, recap the order to ensure everything is correct, check for any additional instructions, and finalize the payment process.
#     The catalog includes:
#     - Stationery: Pens (1.00 each), Notebooks (3.00 each), Sticky Notes (1.50 per pack)
#     - Technology: USB Drive (10.00), Mouse (15.00), Keyboard (20.00)
#     - Furniture: Office Chair (50.00), Desk (100.00), Filing Cabinet (80.00)
#     - Cleaning Supplies: Hand Sanitizer (5.00), Paper Towels (3.00 per roll), Disinfectant Wipes (4.00 per pack)
#     Additional options: Gift Wrapping (+3.00), Next-Day Delivery (+10.00)
#     """,}
# ]

 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
# FINDINGS:

# When the bot gets too much detail or too many options, it can get confused or start making things up.
# Adding unrealistic items led the bot to imagine new items, which disrupted the menu structure.
# Skipping step-by-step guidance also made it miss important parts of the order.